In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
import pandas as pd
import numpy as np

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import json
import networkx as nx
import src.build_graph as bg

In [11]:
import logging
logger = logging.getLogger('debug')
hdlr = logging.FileHandler('/home/zjliao/dsc180a/debug.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.DEBUG)

In [12]:
pwd

'/home/zjliao/dsc180a/notebooks'

In [13]:
logger.debug('testing')

In [5]:
ENV = json.load(open('/home/zjliao/dsc180a/config/env.json'))

In [6]:
cfg = json.load(open('/home/zjliao/dsc180a/config/test-params.json'))
malware = bg.malware_detection(**cfg)
A, apis, apks = malware.buildA_matrix()

get info
check malware
5550
Build A
extract smali
build adjacency matrix


In [8]:
apis

0                              java/lang/Object;-><init>()V
1          veg/mediaplayer/sdk/MediaPlayer;->clearLayout()V
2         android/widget/FrameLayout$LayoutParams;-><ini...
3         java/lang/Integer;->valueOf(I)Ljava/lang/Integer;
4                java/util/Queue;->add(Ljava/lang/Object;)Z
                                ...                        
104072                          net/youmi/android/ct;->c()V
104073    net/youmi/android/bf;-><init>(Lnet/youmi/andro...
104074    net/youmi/android/cu;-><init>(Lnet/youmi/andro...
104075                          net/youmi/android/au;->c()I
104076                          net/youmi/android/ct;->b()V
Length: 104077, dtype: object

In [9]:
A

matrix([[1, 1, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 1, 1, 1]])

In [10]:
B = nx.adjacency_matrix(malware.build_B(), apis)

Build B
create B graph


In [11]:
B

<104077x104077 sparse matrix of type '<class 'numpy.int64'>'
	with 2317188 stored elements in Compressed Sparse Row format>

In [12]:
P = nx.adjacency_matrix(malware.build_P(), apis)

Build P
create P graph


In [13]:
P

<104077x104077 sparse matrix of type '<class 'numpy.int64'>'
	with 1410229 stored elements in Compressed Sparse Row format>

In [14]:
A*A.T

matrix([[30283, 11091, 11958, 13190,     1,   119,   820,   137,   644,
           565],
        [11091, 27927, 11076, 11284,     1,   112,   674,   125,   593,
           510],
        [11958, 11076, 32874, 12366,     1,   116,   859,   133,   561,
           487],
        [13190, 11284, 12366, 46458,     1,   118,  1010,   136,   682,
           580],
        [    1,     1,     1,     1,     1,     1,     1,     1,     1,
             1],
        [  119,   112,   116,   118,     1,   302,    91,   115,    83,
            74],
        [  820,   674,   859,  1010,     1,    91,  5608,   105,   398,
           328],
        [  137,   125,   133,   136,     1,   115,   105,   357,    94,
            75],
        [  644,   593,   561,   682,     1,    83,   398,    94,  2573,
           800],
        [  565,   510,   487,   580,     1,    74,   328,    75,   800,
          1899]])

In [ ]:
A*B*A.T

In [ ]:
A*B*P*P.T*B.T*A.T

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [16]:
x = A
y = [1 if(num<5) else 0 for num in range(10)]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [18]:
def AA(a):
    return a * a.T

def ABA(a, b):
    return a * b * a.T

def APA(a, p):
    return a * p * a.T

def APBPA(a, b, p):
    return a * p * b * p.T * a.T

In [19]:
clf = SVC(kernel = 'precomputed')
kernel_train = AA(X_train)
clf.fit(kernel_train, y_train)
kernel_test = np.dot(X_test, X_train.T)
aa = clf.score(kernel_test, y_test)

In [20]:
# ABA
clf = SVC(kernel = 'precomputed')
kernel_train = ABA(X_train, B)
clf.fit(kernel_train, y_train)
kernel_test = X_test * B * X_train.T
aba = clf.score(kernel_test, y_test)

In [21]:
# APA
clf = SVC(kernel = 'precomputed')
kernel_train = X_train * P * X_train.T
clf.fit(kernel_train, y_train)
kernel_test = X_test * P * X_train.T
apa = clf.score(kernel_test, y_test)

In [22]:
# APBPA
clf = SVC(kernel = 'precomputed')
kernel_train = APBPA(X_train, B, P)
clf.fit(kernel_train, y_train)
kernel_test = X_test * P * B * P.T * X_train.T
apbpa = clf.score(kernel_test, y_test)

In [55]:
results = {
    'aa': aa,
    'aba': aba,
    'apa': apa,
    'apbpa': apbpa
}

In [56]:
_, outpath = ENV

In [57]:
outpath

'output-paths'

In [54]:
res = json.dumps(results)
outpath = ENV['output-paths']
f = open("%s/results.json"%outpath,"w")
f.write(res)
f.close()